###Overview

Look at change insignal power (spectral features) in the 10 Hz and 20 Hz freq bands in the sensorimotor cortex.
    CSP spatial filters trained to enhance signal coming from the brain area. We extract instantaneous power, smooth itand then feed into logistic Reg.


###Preprocessing: 

Band-Pass filtering between 7 and 30 Hz. Then 4 CSP filters applied to signal -> 4 new time series. To train the CSP filters EEG epoched using window of 1.5 seconds before and after the event 'Replace.' CSP training needs 2 classes. Epochs before replace event assumed to contain patterns corresponding to hand movement. Epochs after assumed to contain resting state. We maximize the variance of the signal during hand movement and minimize it during rest.

### Feature Extraction

Preprocessing applied, spatially filtered signal are rectified and convolved with 0.5 second rectangular window for smoothing. Then logarithm applied. Get out a  vector of 4 dimensions for each time sample.

### Classification

For each of the 6 event types logistic regression trained. For training only features downsampled to speed up process. Prediction is probs of the logit reg.

In [2]:
#import utility and validation modules
import utils as utils
import validation as validation

In [7]:
#install basic Python package and dependencies
import numpy as np
import pandas as pd
from glob import glob

utils.print_version('numpy', np)
utils.print_version('pandas', pd)

('numpy: ', '1.9.2')
('pandas: ', '0.16.2')


In [9]:
#Install MNE modules
from mne.io import RawArray
from mne.channels import read_montage
from mne.epochs import concatenate_epochs
from mne import create_info, find_events, Epochs, concatenate_raws, pick_types 
from mne.decoding import CSP

In [14]:
#Install machine learning and signal processing modules
from sklearn.linear_model import LogisticRegression
from scipy.signal import butter, lfilter, convolve, boxcar
from joblib import Parallel, delayed

In [ ]:
def create_raw_mne_object(filename, read_events=True):
    """
    Create mne raw instance from csv file
    """
    data = pd.read_csv(filename)
    
    #Grab channel names
    channel_names = list(data.columns[1:]) #all columns
    
    #Read EEG standard montage (patterns of connection between electrodes; usually 16 or more electrodes)
    """
    Montage means the placement of the electrodes. 
    The EEG can be monitored with either a bipolar montage or a referential one. 
    -Bipolar means that you have two electrodes per one channel, so you have a reference electrode for each channel. 
    -Referential montage means that you have a common reference electrode for all the channels.
    """
    montage = read_montage('standard_1005', channel_names)
    
    #First is EEG (32 channells)
    channel_types = ['eeg']*len(channel_names)
    
    #scale down to microvolts
    data = 1e-6*np.array(data[channel_names]).T
    
    if read_events:
        #get corresponding events_file
        events_filename = filename.replace('_data', '_events')
        events = pd.read_csv(events_filename)
        event_names = events.columns[1:]
        events_data = np.array(events[event_names]).T
        
        #Define channel_types
        #First 32 are EEG
        #Last 6 are the stimulations
        channel_types.extend(['stumulation']*6)
        channel_names.extend(event_names)
        
        #concat data file and events file
        data = np.concatenate((data, events_data))
        
    
    #create and populate MNE info structure
    mne_info = create_info(channel_names, sfreq=500.0, ch_types=channel_types, montage = montage)
    
    mne_info['filename'] = filename
    
    #Make raw array out of mne_info object
    raw = RawArray(data, mne_info, verbose = False)
    return raw     

In [ ]:
def band_pass_filter(raw, picks):
    #Low and high cutoff freqs
    freqs = [7,30]
    #retunrs numerator and denominator polynomials (coefficinets) of the filter
    filterOrder = 5
    criticalFreqs = np.array(freqs)/250.0
    b,a = butter(filterOrder, np.aray(freqs)/250.0, btype='bandpass')
    
    raw._data[picks] = np.array(Parallel(n_jobs=-1)(delayed(lfilter)(b,a,raw._data[i]) for i in picks))
    
    return raw

In [ ]:
def make_epochs(raw_mne, events, picks, stim_channel='Replace'):
    #We are just focused on the 'replace' event so get the position corresponding to that
    #looks it up in our Raw MNE, first 32 events are EEG readings, last 6 are stimuli
    events = find_events(raw_mne, stim_channel, verbose=False)
    
     #epochs signal for 1.5 seconds before the movement
        #will make list of epochs from raw object
        #So we are just defining the epochs by their band around the trigger
        """
        raw: instance of raw MNE info
        events: events that we care about (here we are focused on replace event)
        event_id: id of the event to consider (i think during is how long it lasts which is weird because they should be 2ms)
        time_min: startime before the event (2 ms before here)
        time_max: endtime after the event(i think this is disallowed in the competition)
        proj: whether or not want to apply SSP projection vectors
        """
    epochs = Epochs(raw_mne, events, {'during': 1}, -2, -0.5, proj=False,
                   picks = picks, baseline=None, preload=True,
                   add_eeg_reg=False, verbose=False)
    
    epochs_rest = (raw_mne, events, {'after': 1}, 0.5, 2, proj=False,
                  add_eeg_ref=False, verbose=False)
    
    #workaround to be able to concat epochs with MNE
    epochs_rest.times = epochs.times
    
    return epochs, epochs_rest  

In [ ]:
def rectify_spatially_filtered_signal(raw, csp, picks, num_filters):
    feat = np.dot(csp.filters_[0:nfilters], raw._data[picks])
    feat = feat**2 #power of 2 (squaring!)
    return feat


In [ ]:
def convolve_spatially_filtered_signal(features, csp num_filters):
    testing_features = np.array(Parallel(n_jobs=-1)(delayed(convolve)(feat[i], boxcar(nwin),'full') for i in range(nfilters)))
    testing_features = np.log(testing_features[:, 0:features.shape[1]])
    return testing_features

In [ ]:
def make_training_features(raw, csp, picks, num_filters):
    features = rectify_spatially_filtered_signal(raw, csp, picks, num_features)
    training_features = convolve_spatially_filtered_signal(features, csp, num_features)
    return training_features

In [ ]:
def make_testing_features(raw, csp, picks, num_filters):
    features = rectify_spatially_filtered_signal(raw, csp, picks, num_features)
    testing_features = convolve_spatially_filtered_signal(features, csp, num_features)
    return testing_features

In [ ]:
def make_spatial_features(X, Y, raw_training, raw_testing, picks):
    #csp params
    reg_method = 'lls'
    num_filters = 4
    
    csp = CSP(n_components=num_filters, reg=reg_method)
    csp.fit(X,Y)
    
    testing_features = make_testing_features(raw_training, csp, picks, num_filters)
    training_features = make_training_features(raw_testing, csp, picks, num_filters)
    
    return training_features, testing_features
    

In [ ]:
def make_predictions(training_features, test_features, ids, labels):
    logit = LogisticRegression()
    num_events = 6
    predictions = np.empty(len(ids), num_events) #len(ids) by num_events shape
    for i in range(num_events):
        print('Train subject %d, class %s' % (subject, cols[i]))
        logit.fit(training_features[:, ::subsample].T, labels[i, ::subsample])
        predictions[:, i] = logit.predict_proba(test_features.T)[:,1]
    return predictions 

In [ ]:
#Parameters for main program
subsample_size = 10
subjects = range(1,13)
total_ids = []
total_predictions = []

submission_file = './submissions/baseline.csv'
events =  ['HandStart','FirstDigitTouch',
        'BothStartLoadPhase','LiftOff',
        'Replace','BothReleased']
stimulus_channel = 'Replace'

In [ ]:
#Main Program: Perform analysis for each subject
for subject in subjects:
    total_epochs = []
    y = []
    
    filenames = glob('../input/train/subj%d_series*_data.csv' % (subject))
    testing_filenames = glob('../input/train/subj&d_series*_data.csv' & (subject))
    
    #all the raw mne Arrays concatenated together 
    raw_training = concatenate_raws([create_raw_mne_object(filename) for filename in filenames] )
    raw_testing = concatenate_raws([create_raw_mne_object(filename) for filename in testing_filenames])
    
    
    #pick the channels by type and name
    picks = pick_types(raw.info, eeg=True)
    
    #Filter data for alpha and beta freq bands
    #By default MNE does zero phase (filtfilt) filtering which uses future frames
    #So do left filter to avoid future leakage
    #Parallelized to speed up computations  
    raw_training = band_pass_filter(raw_training, picks)
    raw_testing = band_pass_filter(raw_testing, picks)
    
    #Make Epochs and arrange data
    epochs, epochs_rest = make_epochs(raw_training, events, picks, stimulus_channel)
    total_epochs.append(epochs)
    y.extend([1]*len(epochs))
    y.extend([-1]*len(epochs_rest))
    total_epochs.append(epochs_rest)
    
    #concat all epochs
    epochs = concatenate_epochs(total_epochs)
    
    #get our data
    X = epochs.get_data()
    Y = np.array(Y)
    
    #Makee features with CSP
    training_features, testing_features = make_spatial_features(X, Y, raw_training, raw_testing, picks)
    
    #read ids from testing files
    ids = np.concatenate([np.array(pd.read_csv(filename)['id']) for fname in testing_filenames])
    total_ids.append(ids)
    
    #Get training labels (last 6 channels of raw MNE)
    labels = raw_testing._data[32:]
    
    #Train and predict!
    predictions = make_predictions(training_features, test_features, ids, training_labels)

In [ ]:
#Make submission file
utils.make_submission_file(filename, total_ids, events, predictions)